# Clarke's experiment numerical circuit simulation for the full boxes

## Context

In this notebook we simulate Clarke's experiment (https://doi.org/10.1103/PhysRevA.63.040305) for unambiguous discrimination of single-photon polarization states with the addition of an ancilla photon in one of the input ports. For this purpose we use the *strawberry fields* package. We simulate the experiment for the three boxes.

## Output

- Numerical output photon distributions for the three boxes

In [2]:
import strawberryfields as sf
from strawberryfields.ops import *

import numpy as np
from numpy import pi, sqrt

import matplotlib.pyplot as plt

# set the random seed
np.random.seed(42)

In [14]:
#========= Simulation parameters ============

n_paths = 2
n_modes = n_paths*2
cutoff_dim = 2

#======== Initial State Preparation ==========

# Vacuum State Array
VacuumState = np.zeros([cutoff_dim] * n_modes, dtype='complex')

# One-photon maximum entropy state in path 1 with vertically polarized ancilla photon in path 2
MaxMixedState = np.zeros([cutoff_dim,cutoff_dim] * n_modes, dtype='complex')
MaxMixedState[1,1,0,0,0,0,1,1] = 1/2
MaxMixedState[0,0,1,1,0,0,1,1] = 1/2

MaxMixedState = DensityMatrix(MaxMixedState)

# 2-qubit input state generator
# input photon goes through path 1 in state defined by theta and phi
# ancilla photon goes through path 2 in vertical state
def TwoQubitStateIn(theta, phi):
    State = np.copy(VacuumState)
    
    State[1,0,0,1] = np.cos(theta/2)
    State[0,1,0,1] = np.sin(theta/2)*np.exp(1j*phi)
    
    return State

StateDict = {
    "H" : TwoQubitStateIn(0., 0.),
    "V" : TwoQubitStateIn(pi, 0.),
    "D" : TwoQubitStateIn(pi/2, 0.),
    "A" : TwoQubitStateIn(pi/2, pi),
    "L" : TwoQubitStateIn(pi/2, pi/2),
    "R" : TwoQubitStateIn(pi/2,-pi/2)
}

#============ Gates' Construction =============

SWAP = Interferometer(np.array([[0,1],[1,0]]))

def HWP(angle):
    x = np.cos(2*angle)
    y = np.sin(2*angle)
    return Interferometer(np.array([[x,y],[y,-x]]))

QWPV = Interferometer(np.exp(1j*pi/4)*np.array([[1,0],[0,-1j]]))

#============ Other parameters ===============

alpha = pi/8 # half-angle between states to discriminate
opt_wp4_angle = np.arcsin(np.tan(alpha))/2 # optimal WP4 angle for discrimination

In [15]:
#=========== Circuit construction (for pure states) ============

PureStateProg = sf.Program(n_modes)
InitialState = PureStateProg.params('InitialState')

with PureStateProg.context as q:
    # prepare initial state
    Ket(InitialState) | q

    # apply gates
    QWPV | (q[0], q[1])
    HWP(alpha/2) | (q[0], q[1]) # WP0, for preparation of H and D into "standard" form
    #HWP(pi/2) | (q[0], q[1])
    SWAP | (q[1], q[3]) # PBS2 (Names follow the article convention)
    HWP(opt_wp4_angle) | (q[0], q[1]) # WP4
    HWP(pi/4) | (q[2] , q[3]) # WP5
    SWAP | (q[1], q[3]) # PBS5
    HWP(pi/8) | (q[2], q[3]) # WP6

    # measure
    MeasureFock() | q

#=========== Circuit construction (for maximum entropy state) ============
    
MixedStateProg = sf.Program(n_modes)

with MixedStateProg.context as q:
    # prepare initial state
    MaxMixedState | q

    # apply gates
    QWPV | (q[0], q[1])
    HWP(alpha/2) | (q[0], q[1]) # WP0, for preparation of H and D into "standard" form
    #HWP(pi/2) | (q[0], q[1])
    SWAP | (q[1], q[3]) # PBS2 (Names follow the article convention)
    HWP(opt_wp4_angle) | (q[0], q[1]) # WP4
    HWP(pi/4) | (q[2] , q[3]) # WP5
    SWAP | (q[1], q[3]) # PBS5
    HWP(pi/8) | (q[2], q[3]) # WP6

    # measure
    MeasureFock() | q
    
eng = sf.Engine('fock', backend_options={"cutoff_dim": cutoff_dim})

# Simulation

In [32]:
boxes = ["A", "B", "C"]
PDA = np.array([3,2])/5.
PDB = np.array([2,3])/5.
PDC = np.array([1,1])/2.

def compare_fidelities(freqs): # post measurement protocol to infer which box is given
    FA = np.sqrt(PDA*freqs).sum()**2
    FB = np.sqrt(PDB*freqs).sum()**2
    FC = np.sqrt(PDC*freqs).sum()**2
    inferred_box = boxes[np.argmax([FA,FB,FC])]
    return inferred_box

In [56]:
def execute_sim(N, input_box = "random", trials = 100):
    successes = 0
    for i in range(trials):
        if input_box == "random":
            d = np.random.randint(0,3)
            given_box = boxes[d]
        else:
            given_box = input_box

        samples = []

        if given_box == "A":
            photons = np.array(["H"]*N + ["V"]*N)
        if given_box == "B":
            photons = np.array(["L"]*N + ["R"]*N)

        if given_box == "C":
            for j in range(N):
                result = eng.run(MixedStateProg)
                samples.append(result.samples[0])
        else:
            for s in photons:
                init_state = StateDict[s]
                result = eng.run(PureStateProg, args={'InitialState': init_state})
                samples.append(result.samples[0])

        freqs = np.array(samples).sum(axis=0)[-2:]
        inferred_box = compare_fidelities(freqs)
        if given_box == inferred_box:
            successes += 1

    return 1 - successes/trials

In [65]:
big_trials = 10
box = "A"
trials = 1000
N = 2 # half-number of photons per box
perrs = []

for i in range(big_trials):
    perrs.append(execute_sim(N, box, trials))
    
np.savetxt("data/perr_box{:s}_N{:d}_T{:d}_BT{:d}.txt".format(box, 2*N, trials, big_trials), perrs)